In [6]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
import time

# Define the list of categories (pages)
categories = ["expansion", "masters", "commander", "core"]

# Start WebDriver (using Chrome here)
driver = webdriver.Chrome()

# Initialize lists to store data
expansion_names = []
expansion_codes = []
release_dates = []

# Load the existing CSV if it exists
csv_file = 'expansions_with_codes_and_dates.csv'

if os.path.exists(csv_file):
    # Load the existing CSV into a DataFrame
    df_existing = pd.read_csv(csv_file)
    existing_names = df_existing['Expansion Name'].tolist()  # Get existing expansion names
else:
    # Create an empty DataFrame if no CSV exists
    df_existing = pd.DataFrame(columns=['Expansion Name', 'Expansion Code', 'Release Date'])
    existing_names = []

# Iterate over each category page
for category in categories:
    # Open the page for each category
    url = f'https://scryfall.com/sets?type={category}'
    driver.get(url)

    # Let the page load
    time.sleep(3)

    # Find all expansion names and dates on the page
    name_elements = driver.find_elements(By.CSS_SELECTOR, "table td:nth-child(1) a")
    date_elements = driver.find_elements(By.CSS_SELECTOR, "table td:nth-child(3)")

    # Iterate over each name and date element
    for name_element, date_element in zip(name_elements, date_elements):
        name_full = name_element.text  # Full expansion name including code
        name_without_code = name_full[:-3].strip()  # Name without the last 3 characters
        code = name_full[-3:].strip()  # Last 3 characters (the code)
        
        # Convert the date to a datetime object
        date_str = date_element.text
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')

        # Check if the expansion name is already in the existing CSV
        if name_without_code not in existing_names:
            # If not, add to the lists for new expansions
            expansion_names.append(name_without_code)
            expansion_codes.append(code)
            release_dates.append(date_obj)

# Close the browser once scraping is complete
driver.quit()

# If new expansions are found, append them to the existing DataFrame
if expansion_names:
    # Create a DataFrame for the new expansions
    new_data = {
        'Expansion Name': expansion_names,
        'Expansion Code': expansion_codes,
        'Release Date': release_dates
    }
    df_new = pd.DataFrame(new_data)

    # Append new data to the existing DataFrame
    df_updated = pd.concat([df_existing, df_new], ignore_index=True)

    # Save the updated DataFrame to the CSV
    df_updated.to_csv(csv_file, index=False, encoding='utf-8')

    print(f"{len(expansion_names)} new expansions added and CSV updated.")
else:
    print("No new expansions found. CSV is up to date.")


No new expansions found. CSV is up to date.


In [6]:
import requests
import pandas as pd

# Load expansion data from the CSV
csv_file_path = 'expansions_with_codes_and_dates.csv'
expansions_df = pd.read_csv(csv_file_path)

# Base URL for Scryfall API
base_url = "https://api.scryfall.com/cards/search?q=e%3A"

# Function to get card data from Scryfall API for a given expansion code
def get_card_data(expansion_code):
    url = base_url + expansion_code
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to process card data
def process_card_data(cards, expansion_code):
    card_details = []
    for card in cards:
        # Extract common details
        name = card.get('name', '')
        mana_cost = card.get('mana_cost', '')
        cmc = card.get('cmc', 0)
        color = card.get('colors', [])
        card_type = card.get('type_line', '')
        oracle_text = card.get('oracle_text', '')
        power = card.get('power', '')
        toughness = card.get('toughness', '')
        loyalty = card.get('loyalty', '')
        rarity = card.get('rarity', '')
        is_reserved = card.get('Reserved List', '')

        # Handle double-faced cards
        if card.get('layout') == 'transform':
            # Combine both sides into one entity
            back_face = card.get('card_faces', [])
            if len(back_face) == 2:
                back_name = back_face[1].get('name', '')
                back_oracle_text = back_face[1].get('oracle_text', '')
                oracle_text += f"\n// {back_name}: {back_oracle_text}"
                loyalty = back_face[1].get('loyalty', '')
                rarity = back_face[1].get('rarity', '')
                is_reserved = back_face[1].get('Reserved List', '')
        elif card.get('card_faces'):
            # Separate the two faces if not "transform"
            for face in card['card_faces']:
                face_name = face.get('name', '')
                face_mana_cost = face.get('mana_cost', '')
                face_oracle_text = face.get('oracle_text', '')
                card_details.append({
                    'Expansion Code': expansion_code,
                    'Name': face_name,
                    'Mana Cost': face_mana_cost,
                    'Total Mana Cost': cmc,
                    'Color': face.get('colors', []),
                    'Card Type': face.get('type_line', ''),
                    'Oracle Text': face_oracle_text,
                    'Power': face.get('power', ''),
                    'Toughness': face.get('toughness', ''),
                    'loyalty': face.get('loyalty', ''),
                    'rarity': face.get('rarity', ''),
                    'is_reserved': face.get('Reserved List', '')
                })
            continue
        
        # Append details for single-faced cards
        card_details.append({
            'Expansion Code': expansion_code,
            'Name': name,
            'Mana Cost': mana_cost,
            'Total Mana Cost': cmc,
            'Color': color,
            'Card Type': card_type,
            'Oracle Text': oracle_text,
            'Power': power,
            'Toughness': toughness,
            'loyalty': loyalty,
            'rarity': rarity,
            'is_reserved': is_reserved
        })
    
    return card_details

# Loop through each expansion code and get the card details
all_card_data = []
for _, row in expansions_df.iterrows():
    expansion_name = row['Expansion Name']
    expansion_code = row['Expansion Code']
    print(f"Fetching cards for: {expansion_name} ({expansion_code})")
    
    card_data = get_card_data(expansion_code)
    if card_data and 'data' in card_data:
        processed_data = process_card_data(card_data['data'], expansion_code)
        all_card_data.extend(processed_data)
    else:
        print(f"Failed to fetch data for: {expansion_name} ({expansion_code})")

# Convert the card data into a DataFrame
card_df = pd.DataFrame(all_card_data)

# Append the card data to the original expansion CSV
expanded_csv_file_path = 'expanded_expansions_with_cards.csv'
merged_df = expansions_df.merge(card_df, on='Expansion Code', how='left')

# Save the merged data to a new CSV file
merged_df.to_csv(expanded_csv_file_path, index=False)
print(f"Card details merged and saved to '{expanded_csv_file_path}'")


Fetching cards for: Duskmourn: House of Horror (DSK)
Fetching cards for: Bloomburrow (BLB)
Fetching cards for: Outlaws of Thunder Junction (OTJ)
Fetching cards for: The Big Score (BIG)
Fetching cards for: Murders at Karlov Manor (MKM)
Fetching cards for: The Lost Caverns of Ixalan (LCI)
Fetching cards for: Wilds of Eldraine (WOE)
Fetching cards for: March of the Machine: The Aftermath (MAT)
Fetching cards for: March of the Machine (MOM)
Fetching cards for: Phyrexia: All Will Be One (ONE)
Fetching cards for: The Brothers' War (BRO)
Fetching cards for: Dominaria United (DMU)
Fetching cards for: Streets of New Capenna (SNC)
Fetching cards for: Kamigawa: Neon Dynasty (NEO)
Fetching cards for: Innistrad: Crimson Vow (VOW)
Fetching cards for: Innistrad: Midnight Hunt (MID)
Fetching cards for: Adventures in the Forgotten Realms (AFR)
Fetching cards for: Strixhaven: School of Mages (STX)
Fetching cards for: Kaldheim (KHM)
Fetching cards for: Zendikar Rising (ZNR)
Fetching cards for: Ikoria: La

In [10]:
import pandas as pd

# Load the expanded CSV file
expanded_csv_file_path = 'expanded_expansions_with_cards.csv'
expanded_df = pd.read_csv(expanded_csv_file_path)

# Convert the 'Release Date' column to datetime format for proper sorting
expanded_df['Release Date'] = pd.to_datetime(expanded_df['Release Date'])

# Sort the dataframe by 'Release Date' so that the most recent entries come first
expanded_df = expanded_df.sort_values(by='Release Date', ascending=False)

# Drop duplicates based on card 'Name', keeping the most recent one
cleaned_df = expanded_df.drop_duplicates(subset='Name', keep='first')

# Save the cleaned data to a new CSV file
cleaned_csv_file_path = 'cleaned_expansions_with_cards.csv'
cleaned_df.to_csv(cleaned_csv_file_path, index=False)

print(f"Duplicates removed. Cleaned data saved to '{cleaned_csv_file_path}'")


Duplicates removed. Cleaned data saved to 'cleaned_expansions_with_cards.csv'


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configuração do WebDriver (use o caminho para o driver correto, por exemplo: chromedriver.exe)
driver = webdriver.Chrome()

# Função para fazer login no site melee.gg
def login_melee(driver, username, password):
    driver.get('https://melee.gg/Account/SignIn')  # URL de login
    time.sleep(3)
    
    # Encontre os campos de usuário e senha e faça login
    # Encontrar o campo de e-mail e preenchê-lo
    email_input = driver.find_element(By.NAME, 'Email')
    email_input.send_keys(username)
    email_input.send_keys(Keys.RETURN)
    
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, 'Password')))
    # Encontrar o campo de senha e preenchê-lo
    password_input = driver.find_element(By.NAME, 'Password')
    password_input.send_keys(password)
    
    # Enviar o formulário de login
    password_input.send_keys(Keys.RETURN)
    
    # Espera para garantir que o login foi realizado
    time.sleep(5)

# Função para buscar uma carta e capturar as estatísticas
def search_card(driver, card_name, start_date, end_date):
    # Modificar a URL de busca com o nome da carta e datas
    url = f"https://melee.gg/Decklist/Search?Cards%5B%5D={card_name}&Date%5BStart%5D={start_date}&Date%5BEnd%5D={end_date}"
    driver.get(url)
    
    # Espera para a página carregar
    time.sleep(5)

    # Exemplo: Capturando as estatísticas de Win/Loss/Draw (ajuste conforme a estrutura do site)
    # Isso depende de como os resultados são mostrados na página
    try:
        card_result = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[5]').text  # Ajustar conforme o site
        win = card_result[0]
        loss = card_result[2]
        draw = card_result[4]
    except:
        if (driver.find_element(By.XPATH, '//*[@id="search-table"]/tbody/tr/td').text == 'Não foram encontrados decklists'):
            win = 0
            loss = 0
            draw = 0
    
    return {
        'card': card_name,
        'wins': win,
        'losses': loss,
        'draws': draw
    }

# Função principal
def main():
    # Caminho para o CSV com os nomes das cartas
    csv_file_path = 'cleaned_expansions_with_cards.csv'
    df = pd.read_csv(csv_file_path)
    
    # Informações de login
    username = 'rodrigo.duarte2@estudante.ufla.br'
    password = 'Sonin123?'
    
    # Fazer login no site
    login_melee(driver, username, password)
    
    # Período de datas (exemplo)
    start_date = '2020-01-01T00:00:00.000Z'
    end_date = '2024-08-28T23:59:59.999Z'
    
    results = []
    
    # Iterar sobre cada carta no CSV
    for card_name in df['Name']:
        card_stats = search_card(driver, card_name, start_date, end_date)
        results.append(card_stats)
    
    # Converter os resultados para um DataFrame e salvar como CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv('card_stats_results.csv', index=False)
    
    # Fechar o driver após completar as buscas
    driver.quit()

# Executar o script
if __name__ == "__main__":
    main()


UnboundLocalError: cannot access local variable 'draw' where it is not associated with a value

In [27]:
df = pd.read_csv('cleaned_expansions_with_cards.csv')

df = df[df['Card Type'] != 'Scheme']

df.to_csv('cleaned_expansions_with_cards.csv')


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
import time
import pandas as pd

from tqdm import tqdm

# Iniciar o WebDriver com as opções configuradas
driver = webdriver.Edge()

# Carregar o arquivo CSV contendo as URLs das cartas
csv_path = 'updated_merged_cleaned_with_expansions.csv'
df = pd.read_csv(csv_path)
# df['Name'] = df['Name'].apply(lambda x: x.split('//')[0].strip())

df = df[df.Price1 == 'Erro']
print(df.value_counts('Name'))
# Construa a URL base para cada carta (substituir NaN e filtrar valores ausentes)
base_url = "https://www.ligamagic.com.br/?view=cards/card&card="
clean_card_names = df['Name'].dropna().unique()

# Construa as URLs para pares de cartas
card_urls2 = {}
for i in range(0, len(clean_card_names), 2):
    if i + 1 < len(clean_card_names):
        card_name_pair = f"{clean_card_names[i]} %20//%20 {clean_card_names[i + 1]}"
        card_url_pair = base_url + card_name_pair.replace(' ', '%20')
        card_urls2[card_name_pair] = card_url_pair
# Função para selecionar a expansão e capturar os preços
def get_card_prices(card_url):
    try:
        driver.get(card_url)  # Navegar até a URL da carta 


        try:
            minimun = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[3]/div[2]/div/div[2]')
            minimun = minimun.get_attribute('innerText')

            medium = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[3]/div[2]/div/div[4]')
            medium = medium.get_attribute('innerText')

            maximun = driver.find_element(By.XPATH, '/html/body/main/div[4]/div[1]/div/div[3]/div[2]/div/div[6]')
            maximun = maximun.get_attribute('innerText')
        
        except:
            minimun = 'Erro'
            medium = 'Erro'
            maximun = 'Erro'

        return [minimun, medium, maximun]


    except:
        pass
                
        return [minimun, medium, maximun]
        
        

# Coletar todos os dados de preços das cartas
card_price_data = []  # List to store card data

for card_name, card_url in tqdm(card_urls2.items(), desc="Processing Cards", unit="card"):
        
        # Call your function to get the card prices
        prices = get_card_prices(card_url)
        
        # Append the card name and its prices to the card_price_data list
        card_price_data.append([card_name] + prices)


# Fechar o WebDriver
driver.quit()

# Salvar os preços em um DataFrame
price_df = pd.DataFrame(card_price_data, columns=["Name", "Price1", "Price2", "Price3"])

# Salvar os resultados em um arquivo CSV
price_df.to_csv("card_prices.csv", index=False)

# Exibir os primeiros resultados
price_df.head()


Name
Akki Lavarunner    1
Alive              1
Appeal             1
Armed              1
Assault            1
                  ..
Turn               1
Victory            1
Wear               1
Well               1
Willing            1
Name: count, Length: 144, dtype: int64


Processing Cards:  12%|█▎        | 9/72 [00:12<01:27,  1.39s/card]


UnboundLocalError: cannot access local variable 'minimun' where it is not associated with a value

In [43]:
card_price_data 

[['Cease', 'Erro', 'Erro', 'Erro'],
 ['Concoct', 'Erro', 'Erro', 'Erro'],
 ['Tail the Suspect', 'Erro', 'Erro', 'Erro'],
 ['Hustle', 'Erro', 'Erro', 'Erro'],
 ['Bother', 'Erro', 'Erro', 'Erro'],
 ['Assemble', 'Erro', 'Erro', 'Erro'],
 ['Bedeck', 'Erro', 'Erro', 'Erro'],
 ['Expansion', 'Erro', 'Erro', 'Erro'],
 ['Explosion', 'Erro', 'Erro', 'Erro'],
 ['Memory', 'Erro', 'Erro', 'Erro'],
 ['Coward', 'Erro', 'Erro', 'Erro'],
 ['Killer', 'Erro', 'Erro', 'Erro'],
 ['Go Hog Wild', 'Erro', 'Erro', 'Erro'],
 ['Ice', 'Erro', 'Erro', 'Erro'],
 ['Reality', 'Erro', 'Erro', 'Erro'],
 ['Battery', 'Erro', 'Erro', 'Erro'],
 ['Assault', 'Erro', 'Erro', 'Erro'],
 ['Suffering', 'Erro', 'Erro', 'Erro'],
 ['Fire', 'Erro', 'Erro', 'Erro'],
 ['Incubation', 'Erro', 'Erro', 'Erro'],
 ['Bust', 'Erro', 'Erro', 'Erro'],
 ['Boom', 'Erro', 'Erro', 'Erro'],
 ['Beyeen Coast', 'Erro', 'Erro', 'Erro'],
 ['Oblivion', 'Erro', 'Erro', 'Erro'],
 ['Consign', 'Erro', 'Erro', 'Erro'],
 ['Fame', 'Erro', 'Erro', 'Erro'],
 ['Clai

In [44]:
import pandas as pd

df = pd.read_csv('updated_merged_cleaned_with_expansions.csv')
df2 = pd.read_csv('card_prices.csv')

In [45]:
# Merge DataFrames on the 'Name' column, updating prices in df with those from df2
# Handle cases where df has names with ' // ' by splitting and matching with df2
# df['Name'] = df['Name'].apply(lambda x: x.split(' // ')[0].strip())
updated_df = pd.merge(df, df2, on='Name', how='left', suffixes=('', '_new'))
updated_df.drop_duplicates(subset='Name', keep='first', inplace=True)
# Update the price columns in df with the new prices from df2
updated_df['Price1'] = updated_df['Price1_new'].combine_first(updated_df['Price1'])
updated_df['Price2'] = updated_df['Price2_new'].combine_first(updated_df['Price2'])
updated_df['Price3'] = updated_df['Price3_new'].combine_first(updated_df['Price3'])

# Drop the temporary new price columns
updated_df.drop(columns=['Price1_new', 'Price2_new', 'Price3_new'], inplace=True)

# Save the updated DataFrame to a new CSV file
updated_df.to_csv('updated_merged_cleaned_with_expansions.csv', index=False)

# Display the first few rows of the updated DataFrame
updated_df.head()

,Unnamed: 0.1,Unnamed: 0,Expansion Name,Expansion Code,Release Date,Name,Mana Cost,Total Mana Cost,Color,Card Type,Oracle Text,Power,Toughness,Price1,Price2,Price3
0,0,1,Duskmourn: House of Horror Commander,DSC,2024-09-27,Access Maze,{5}{B}{B},11.0,[],Enchantment — Room,(You may cast either half. That door unlocks o...,NaN,NaN,"R$ 4,94","R$ 8,62","R$ 19,99"
1,1,2,Duskmourn: House of Horror Commander,DSC,2024-09-27,Cramped Vents,{3}{B},11.0,[],Enchantment — Room,(You may cast either half. That door unlocks o...,NaN,NaN,"R$ 4,94","R$ 8,62","R$ 19,99"
2,2,3,Duskmourn: House of Horror Commander,DSC,2024-09-27,Counterspell,{U}{U},2.0,['U'],Instant,Counter target spell.,NaN,NaN,"R$ 7,49","R$ 9,46","R$ 14,50"
3,3,4,Duskmourn: House of Horror Commander,DSC,2024-09-27,Convert to Slime,{3}{B}{G},5.0,"['B', 'G']",Sorcery,"Destroy up to one target artifact, up to one t...",NaN,NaN,"R$ 4,89","R$ 9,62","R$ 24,94"
4,4,5,Duskmourn: House of Horror Commander,DSC,2024-09-27,Command Tower,NaN,0.0,[],Land,{T}: Add one mana of any color in your command...,NaN,NaN,"R$ 0,94","R$ 1,59","R$ 3,99"


In [32]:
df.to_csv('merged_cleaned_with_expansions.csv')

In [31]:
# Merge DataFrames on the 'ID' column
merged_df = pd.merge(df, df2, on='Name')

print(merged_df)

merged_df.to_csv('merged_cleaned_with_expansions.csv')

      Unnamed: 0.1  Unnamed: 0                        Expansion Name  \
0              143         152  Duskmourn: House of Horror Commander   
1              186         195            Duskmourn: House of Horror   
2              274         283            Duskmourn: House of Horror   
3              322         331            Duskmourn: House of Horror   
4              323         332            Duskmourn: House of Horror   
...            ...         ...                                   ...   
5353         18983       18993                             Alliances   
5354         18984       18994                             Alliances   
5355         18985       18995                             Alliances   
5356         18986       18996                             Alliances   
5357         18987       18997                             Alliances   

     Expansion Code Release Date                            Name  Mana Cost  \
0               DSC   2024-09-27                  Evolvi

In [2]:
import pandas as pd

csv_path = 'updated_merged_cleaned_with_expansions.csv'
df = pd.read_csv(csv_path)
df[df.Price1 == 'Erro']

,Unnamed: 0.1,Unnamed: 0,Expansion Name,Expansion Code,Release Date,Name,Mana Cost,Total Mana Cost,Color,Card Type,Oracle Text,Power,Toughness,Price1,Price2,Price3
497,497,506,Bloomburrow Commander,BLC,2024-08-02,Dusk,{2}{W}{W},9.0,[],Sorcery,Destroy all creatures with power 3 or greater.,NaN,NaN,Erro,Erro,Erro
498,498,507,Bloomburrow Commander,BLC,2024-08-02,Dawn,{3}{W}{W},9.0,[],Sorcery,Aftermath (Cast this spell only from your grav...,NaN,NaN,Erro,Erro,Erro
689,689,698,Modern Horizons 3 Commander,M3C,2024-06-14,Find,{B/G}{B/G},8.0,[],Sorcery,Return up to two target creature cards from yo...,NaN,NaN,Erro,Erro,Erro
693,693,702,Modern Horizons 3 Commander,M3C,2024-06-14,Finality,{4}{B}{G},8.0,[],Sorcery,You may put two +1/+1 counters on a creature y...,NaN,NaN,Erro,Erro,Erro
839,839,848,Outlaws of Thunder Junction Commander,OTC,2024-04-19,Heaven,{X}{G},3.0,[],Instant,Heaven deals X damage to each creature with fl...,NaN,NaN,Erro,Erro,Erro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15959,15959,15969,Champions of Kamigawa,CHK,2004-10-01,"Autumn-Tail, Kitsune Sage",NaN,4.0,[],Legendary Creature — Fox Wizard,{1}: Attach target Aura attached to a creature...,4,5,Erro,Erro,Erro
15971,15971,15981,Champions of Kamigawa,CHK,2004-10-01,Bushi Tenderfoot,{W},1.0,[],Creature — Human Soldier,When a creature dealt damage by Bushi Tenderfo...,1,1,Erro,Erro,Erro
15972,15972,15982,Champions of Kamigawa,CHK,2004-10-01,Kenzo the Hardhearted,NaN,1.0,[],Legendary Creature — Human Samurai,Double strike; bushido 2 (Whenever this creatu...,3,4,Erro,Erro,Erro
15987,15987,15997,Champions of Kamigawa,CHK,2004-10-01,Akki Lavarunner,{3}{R},4.0,[],Creature — Goblin Warrior,Haste\r\nWhenever Akki Lavarunner deals damage...,1,1,Erro,Erro,Erro
